<a href="https://colab.research.google.com/github/DrSubbiah/1.Bayesian-Inference/blob/master/12_Interpretation_Proportion_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="darkblue">Binary Data Model

In [ ]:
import numpy as np
import pandas as pd
import statistics as stat
import scipy
import pystan

In [ ]:
#For plots
import arviz as az
import matplotlib.pyplot as plt

#<font color="darkblue"> Loading Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
path = "/content/drive/MyDrive/Data Sets/Heart_data.csv"
hrt_da = pd.read_csv(path)

In [ ]:
hrt_da

,row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,1,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,Yes
1,2,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,Yes
2,3,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,No
3,4,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,Yes
4,5,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,Yes
...,...,...,...,...,...,...,...,...,...,...,...
457,459,214,0.40,5.98,31.72,Absent,64,28.45,0.00,58,No
458,460,182,4.20,4.41,32.10,Absent,52,28.61,18.72,52,Yes
459,461,108,3.00,1.59,15.23,Absent,40,20.09,26.64,55,No
460,462,118,5.40,11.61,30.79,Absent,64,27.35,23.97,40,No


# <font color="darkblue"> Odds calculation

For the response variable (binary) odds is defined as

$$\frac {\mathrm{success}}{\mathrm{failure}}$$

In [ ]:
#pd.DataFrame(hrt_da.chd.value_counts()).T

daf1=pd.DataFrame(hrt_da.chd.value_counts()).T
daf2=daf1.reindex(['Yes', 'No'], axis="columns")
daf2

,Yes,No
chd,160,302


In [ ]:

odr=round(daf2.iloc[0,0]/daf2.iloc[0,1],4)
lodr=round(np.log(odr),4)
print(odr,lodr)

0.5298 -0.6353


In [ ]:
hrt_da=hrt_da.assign(chd_c=lambda x:x['chd'].apply(lambda y: 1 if y=="Yes" else 0))

In [ ]:
hrt_da['chd_c'] = hrt_da['chd_c'].astype('category')

In [ ]:
print(hrt_da.dtypes)

# Input - data and values for prior parameters a and b

In [ ]:
hrt_data = {
             'n': len(hrt_da),
             'y': len(hrt_da.loc[hrt_da['chd'] == 'Yes']),
             'a':1,
             'b':1,
            }
print(hrt_data)

{'n': 462, 'y': 160, 'a': 1, 'b': 1}


In [ ]:
hrt_code1 = """
data {
    real<lower=0> a;
    real<lower=0> b;
    int<lower=0> n;
    int<lower=0> y;
}

parameters {
    real<lower=0, upper=1> p;
}

transformed parameters {
  real theta;
  theta=logit(p);
}

model {
      y ~ binomial(n, p);
      p ~ beta(a, b);
}
"""
# posterior
posterior1 = pystan.StanModel(model_code=hrt_code1)

In [ ]:
fit_model1= posterior1.sampling(data=hrt_data,
                  iter=10000,
                  chains=4,
                  seed=1,
                  warmup=3000,
                  thin=1,
                  control={"max_treedepth":15,"adapt_delta" : 0.9999})

# <font color="darkorange">Pystan Summary - Information about Model and Metrics

In [ ]:
fit_model1_summary=fit_model1.stansummary(pars=["p","theta"], probs=(0.025, 0.25, 0.5, 0.75, 0.975), digits_summary=4)
print(fit_model1_summary)

# <font color="darkorange"> Condensed Summary Report

In [ ]:
summ_mod1=az.summary(fit_model1,round_to=3,hdi_prob=0.95)
summ_mod1

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
p,0.347,0.022,0.304,0.389,0.000,0.000,7249.863,7066.032,1.001
theta,-0.636,0.097,-0.830,-0.450,0.001,0.001,7249.863,7066.032,1.001


# <font color="darkgreen">Inverse Transformation

In this model logit transform has been used to find the log of odds for success.

Appropriate Inverse transformation function can be used to find the probability from logit

## logit from probability (p)

$$\mathrm{y=logit}(p) = \log(\frac{p}{1-p})$$

## Odds

$$e^y=\frac{p}{1-p}$$

## Inverse logit - expit(y)

$$\mathrm{p=expit}(y) = \frac{e^y}{1+e^y}=\frac{1}{1+e^{-y}}$$

In [ ]:
theta_op=summ_mod1.iloc[1,[0,2,3]]
scipy.special.expit(theta_op)

#<font color="darkorange"> MCMC convergence Diagnostics - Plots

In [ ]:
az.plot_trace(fit_model1, compact=False,legend=True)
plt.show()

# <font color="darkorange">Histogram, Density plot from the posterior1 for the QoI

In [ ]:
# QoI "p"
az.plot_dist(fit_model1['p'],quantiles=[.25, .5, .75],kind="hist",figsize=(20, 6))
plt.show()

In [ ]:
#QoI "theta=logit(p)"
az.plot_dist(fit_model1['theta'],quantiles=[0.25, 0.5, 0.75],kind="kde",figsize=(20, 6))
plt.show()

# <font color="darkgreen">Posterior Probabilities

1. Obtain the generated samples at each iteration

  - Use pystan function *extract*

2. Compute the required probabilites as a ratio of favourable points (desired condtion) to the total number of samples

In [ ]:
zz=pd.DataFrame(fit_model1.extract())

In [ ]:
pr1=np.count_nonzero(zz["p"]> 0.3, axis=0)/len(zz)

pr2=np.count_nonzero(zz["p"]< 0.345, axis=0)/len(zz)

pr3=np.count_nonzero((zz["p"]>0.3) & (zz["p"] < 0.345))/len(zz)

print(round(pr1,4),round(pr2,4),round(pr3,4))

# Binomial Regression without predictor



In [ ]:
hrt_fit_code0 = """
data {
    real<lower=0> a;
    real<lower=0> b;
    int<lower=0> k;

    int y[k];
}

parameters {
  real b0;
 }

model {
     y ~ bernoulli_logit(b0);
     b0 ~ normal(a, b);
}
"""


# Posterior
posterior0 = pystan.StanModel(model_code=hrt_fit_code0)


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_3927b8f5c85f6b45d8fbd9f759a9506f NOW.


In [ ]:
# Input - data and values for prior parameters a and b

n=len(hrt_da)
y1=hrt_da['chd_c']
#y2=print((y1.to_numpy()))
hrt_data0 = {'k':n,
             'y':y1,
             'a':0,
             'b':1,
             }


In [ ]:
hrt_fit0= posterior0.sampling(data=hrt_data0,
                  iter=10000,
                  chains=4,
                  seed=1,
                  warmup=3000,
                  thin=1,
                  control={"max_treedepth":15,"adapt_delta" : 0.9999})

print(hrt_fit0.stansummary(pars=["b0"], probs=(0.025, 0.25, 0.5, 0.75, 0.975), digits_summary=4))

In [ ]:
summ_fit0=az.summary(hrt_fit0,round_to=3,hdi_prob=0.95)
summ_fit0

In [ ]:
az.plot_trace(hrt_fit0,compact=False, legend=True)
plt.show()

In [ ]:
# QoI "p"
az.plot_dist(hrt_fit0['b0'],quantiles=[.25, .5, .75],kind="hist",figsize=(20, 6))
plt.show()

# Binary Logit Model with One Binary Predictor

In [ ]:
hrt_fit_code1 = """
data {
int<lower=0> N;
int<lower=0,upper=1> y[N];
vector[N] x;
}

parameters {
real b0;
real b1;
}
model {
y ~ bernoulli_logit(b0 + b1 * x);

b0 ~ normal(0,100);
b1 ~ normal(0,100);
}
"""


# Posterior
posterior_fit1 = pystan.StanModel(model_code=hrt_fit_code1)


NameError: ignored

In [ ]:
hrt_da=hrt_da.assign(famhist_c=lambda x:x['famhist'].apply(lambda y: 1 if y=="Present" else 0))
hrt_da['famhist_c'] = hrt_da['famhist_c'].astype('category')

In [ ]:
# Input - data and values for prior parameters a and b

n=len(hrt_da)
y1=hrt_da['chd_c']
x1=hrt_da['famhist_c']
#y2=print((y1.to_numpy()))
hrt_data1 = {'N':n,
             'y':y1,
             'x':x1,
            }
hrt_data1

In [ ]:
hrt_fit1= posterior_fit1.sampling(data=hrt_data1,
                  iter=10000,
                  chains=4,
                  seed=1,
                  warmup=3000,
                  thin=1,
                  control={"max_treedepth":15,"adapt_delta" : 0.9999})

# print(hrt_fit1.stansummary(pars=["b0"], probs=(0.025, 0.25, 0.5, 0.75, 0.975), digits_summary=4))

In [ ]:
summ_fit1=az.summary(hrt_fit1,round_to=3,hdi_prob=0.95)
summ_fit1

In [ ]:
#Cross tabulation
df1=pd.crosstab(hrt_da['famhist_c'],hrt_da['chd_c'],margins = False)
df2=df1.reindex(['1', '0'], axis="columns").reindex(['1', '0'])
df2

In [ ]:
a=df2.loc[1,1]
b=df2.loc[1,0]
c=df2.loc[0,1]
d=df2.loc[0,0]
oddsratio=(a*d)/(b*c)
logoddsratio=np.log(oddsratio)

print(oddsratio,logoddsratio)

In [ ]:
az.plot_trace(hrt_fit1,compact=False, legend=True)
plt.show()

In [ ]:
az.plot_posterior(hrt_fit1,
            var_names=['b0', 'b1'],
            transform=scipy.special.expit,
            hdi_prob=0.95,
            kind="hist")
plt.show()